# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Introduction

Toronto is a big city and Canada's biggest city. Toronto is considered one of the most culturally diverse cities in the world, with more than half the population born outside of the city

I want to explore and extract the similarity and disimilarity of each area (Postal Code) in Toronto that can help tourists who want to visit Toronto by providing insight.

## DATA

I will use machine learning technique to segment the area which are similar based on
* number of each existing Venue Category in the area within 500 meter from Center Point of the area

In [ ]:
import pandas as pd
import requests
import folium

pd.set_option('display.max_rows', None)

### Preprocess Data
* Toronto data from Wikipedia
* latitude & longitude coordinates for Toronto's Postal Code
* Clean data

In [23]:
Toronto = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

# Drop `NaN` and `Not assigned`
Toronto = Toronto[ Toronto['Borough'].notna() & (Toronto['Borough'] != 'Not assigned') ]
# Reset Index
Toronto = Toronto.reset_index(drop=True)
# If Neighborhood is Nan, fill with Borough
Toronto.Neighborhood.fillna(Toronto.Borough, inplace=True)

LatLongToronto = pd.read_csv("https://cocl.us/Geospatial_data")

Toronto = Toronto.set_index('Postal Code').join( LatLongToronto.set_index('Postal Code') ).reset_index()

Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Map

In [43]:
# Toronto
latitude = 43.651070
longitude = -79.347015
TorontoMap = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Postal Code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(TorontoMap)  
    
TorontoMap

In [33]:
CLIENT_ID = 'IKU0SZMHOXR45B42XVDJU1QWCDNMFBCZSPEQ5QUZUK13IMHX'
CLIENT_SECRET = 'JG25RHSYJG4XU4JTKBV3C3LW30NWLRJL5G4Q0XE4N2RNJX2N'
VERSION = '20200625'
LIMIT = 100
radius = 500

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append(
            [(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results
            ])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [ 
        'District', 
        'District Latitude', 
        'District Longitude', 
        'Venue', 
        'Venue Latitude', 
        'Venue Longitude', 
        'Venue Category'
    ]
    
    return nearby_venues

## DATA EXPLORATION

In [41]:
TorontoVenue = getNearbyVenues( names=Toronto['Postal Code'], latitudes=Toronto['Latitude'], longitudes=Toronto['Longitude'] )

In [53]:
TorontoVenue.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M3A,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [60]:
TorontoVenue["Venue Category"].unique()

array(['Park', 'Food & Drink Shop', 'Construction & Landscaping',
       'Hockey Arena', 'Portuguese Restaurant', 'Coffee Shop',
       'Intersection', 'Pizza Place', 'Bakery', 'Distribution Center',
       'Spa', 'Pub', 'Restaurant', 'Breakfast Spot',
       'Gym / Fitness Center', 'Historic Site', 'Farmers Market',
       'Performing Arts Venue', 'Chocolate Shop', 'Dessert Shop',
       'French Restaurant', 'Café', 'Yoga Studio', 'Theater',
       'Event Space', 'Shoe Store', 'Ice Cream Shop', 'Art Gallery',
       'Cosmetics Shop', 'Asian Restaurant', 'Electronics Store', 'Bank',
       'Beer Store', 'Antique Shop', 'Boutique', 'Furniture / Home Store',
       'Vietnamese Restaurant', 'Clothing Store', 'Accessories Store',
       "Women's Store", 'Miscellaneous Shop', 'Arts & Crafts Store',
       'Athletics & Sports', 'Italian Restaurant', 'Sushi Restaurant',
       'Creperie', 'Beer Bar', 'Burrito Place', 'Mexican Restaurant',
       'Hobby Shop', 'Diner', 'Fried Chicken Joint', '

In [62]:
TorontoVenue["Venue Category"].value_counts()

Coffee Shop                        174
Café                                98
Restaurant                          71
Park                                51
Pizza Place                         49
Italian Restaurant                  45
Bakery                              45
Sandwich Place                      42
Hotel                               41
Japanese Restaurant                 39
Clothing Store                      34
Gym                                 33
Sushi Restaurant                    30
Bar                                 29
Grocery Store                       29
Pub                                 26
American Restaurant                 26
Fast Food Restaurant                26
Bank                                25
Breakfast Spot                      22
Pharmacy                            22
Thai Restaurant                     21
Seafood Restaurant                  20
Diner                               19
Ice Cream Shop                      19
Bookstore                

In [80]:
TorontoGroup = TorontoVenue.groupby(['District', 'Venue Category']).size().reset_index(name='Counts')
TorontoGroup.sort_values(by=['District', 'Counts'], inplace=True, ascending=False)
TorontoGroup

,District,Venue Category,Counts
1528,M9W,Bar,1
1529,M9W,Drugstore,1
1530,M9W,Rental Car Location,1
1524,M9V,Grocery Store,2
1526,M9V,Pizza Place,2
1521,M9V,Beer Store,1
1522,M9V,Fast Food Restaurant,1
1523,M9V,Fried Chicken Joint,1
1525,M9V,Pharmacy,1
1527,M9V,Sandwich Place,1


## Conclusion

These are the most common veune in Toronto.
* Coffee Shop
* Restaurant
* Park
* Pizza place
* Italian Restaurant